In [16]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from statistics import mean
from sklearn.metrics import confusion_matrix
import math
import datetime


#reading and cleaning the data to train our model
data = pd.read_csv('Algerian_forest_fires.csv', usecols = ["temp","rel_hum","wind_sp","rain", "FFMC", "DMC", "DC", "ISI", "BUI", "FWI", "class"])
data['class'] = data['class'].str.strip()


#seperating the features and label
X = data[["temp","rel_hum","wind_sp","rain", "FFMC", "DMC", "DC", "ISI", "BUI", "FWI"]]
y = data['class']

#training our model and doing cross validation to get accuracy and f-score
a_list = list()
f_list = list()
p_list = list()
r_list = list()
#doing 100 fold validation to get average accuracy of model
for i in range(0,100):
    #splitting the data into train and test set
    x_train, x_test, y_train, y_test = train_test_split(X, y)
    
    #Non Linear SVM with polynomial kernal(degree = 2) has been used
    SVC_class = svm.NuSVC(kernel = 'poly', degree = 2, probability = True)
    SVC_class.fit(x_train, y_train)

    #predicting labels for test set
    y_pred = SVC_class.predict(x_test)
    
    #generating confusion matrix to get the precision and recall
    a = confusion_matrix(y_test, y_pred)
    r = a[0][0]/(a[0][0] + a[0][1])
    p = a[0][0]/(a[0][0] + a[1][0])
    
    #calculating the f-score
    f_score = 2 * (p * r) / (p + r)

    p_list.append(p)
    r_list.append(r)
    f_list.append(f_score)
    
    # getting the accuracy score of the model
    a_list.append(accuracy_score(y_test, y_pred))
    
#printing the accuracy and f-score of the model
print("Accuracy", mean(a_list) * 100)
print("F-Score",  mean(f_list))

Accuracy 90.75409836065573
F-Score 0.9206680792656041


In [17]:
#this cell demonstrates how the model works and how the labels are distributed into 5 calsses from extremly high probability of
#fire to very low probability

x_train, x_test, y_train, y_test = train_test_split(X, y)
SVC_class = svm.NuSVC(kernel = 'poly', degree = 2, probability = True)
SVC_class.fit(x_train, y_train)
y_test = y_test.reset_index()
x_test = x_test.reset_index()
x_test.drop('index', inplace = True, axis=1)
y_test.drop('index', inplace = True, axis=1)
for i in range(0,11):
    r = np.random.randint(0,len(y_test))
    y_pred = SVC_class.predict_proba(np.array(x_test.loc[r]).reshape(1,-1))
    print(x_test.loc[r])
    p = y_pred[0][0]
    print('predicted probability of fire = ', p)
    
    #below the probabilities are mapped into 5 classes
    if p > 0.80:
        print("very high probability of fire")
        print("\n\n")
    elif p > 0.60:
        print("high probability of fire")
        print("\n\n")
    elif p > 0.40:
        print("moderate probability of fire")
        print("\n\n")
    elif p > 0.20:
        print("low probability of fire")
        print("\n\n")
    else:
        print("very low probability of fire")
        print("\n\n")

temp       29.0
rel_hum    75.0
wind_sp    16.0
rain        0.0
FFMC       80.8
DMC         3.4
DC         24.0
ISI         2.8
BUI         5.1
FWI         1.7
Name: 46, dtype: float64
predicted probability of fire =  0.35452589865846706
low probability of fire



temp       32.0
rel_hum    63.0
wind_sp    14.0
rain        0.0
FFMC       87.0
DMC        10.9
DC         37.0
ISI         5.6
BUI        12.5
FWI         6.8
Name: 2, dtype: float64
predicted probability of fire =  0.8890795083343066
very high probability of fire



temp       34.0
rel_hum    59.0
wind_sp    16.0
rain        0.0
FFMC       88.1
DMC        19.5
DC         47.2
ISI         7.4
BUI        19.5
FWI        10.9
Name: 31, dtype: float64
predicted probability of fire =  0.9747456659657293
very high probability of fire



temp       33.0
rel_hum    73.0
wind_sp    12.0
rain        1.8
FFMC       59.9
DMC         2.2
DC          8.9
ISI         0.7
BUI         2.7
FWI         0.3
Name: 41, dtype: float64
predicted p

In [20]:
#this segment of the code contains methods to caculate the forest fire weather indices

cffdrs_el = [6.5,7.5,9.0,12.8,13.9,13.9,12.4,10.9,9.4,8.0,7.0,6.0]
cffdrs_fl = [-1.6,-1.6,-1.6,0.9,3.8,5.8,6.4,5.0,2.4,0.4,-1.6,-1.6]
    
#function to calculate ffmc(forest fuel moisture code)
def _ffmc(temps, rhs, wss, rains, ffmc_old):
    wmo = 147.2*(101.0-ffmc_old)/(59.5+ffmc_old)
    if rains > 0.5:
        ra = rains - 0.5
        if wmo > 150.0:
            wmo_add = 42.5*ra*math.exp(-100.0/(251.-wmo))*(1.0-math.exp(-6.93/ra))+0.0015*(wmo-150.)*(wmo-150.)*math.sqrt(ra)
        else:
            wmo_add = 42.5*ra*math.exp(-100.0/(251.-wmo))*(1.0-math.exp(-6.93/ra))
        wmo += wmo_add

    if wmo > 250.0:
        wmo = 250.0
    ed = 0.942 * (rhs ** 0.679) + (11.0 * math.exp((rhs-100.0)/10.0)) + 0.18 * (21.1-temps) * (1.0 - 1.0 / math.exp(rhs*0.115))
    ew = 0.618 * (rhs ** 0.753) + (10.0 * math.exp((rhs-100.0)/10.0)) + 0.18 * (21.1-temps) * (1.0 - 1.0 / math.exp(rhs*0.115))

    wm = wmo

    if wmo < ed and wmo < ew:
        z = 0.424 * ( 1.0 - ((100.0-rhs)/100.0) ** 1.7) + 0.0694 * math.sqrt(wss) * (1.0 - ((100.0-rhs)/100.0) ** 8.0)
        x = z * 0.0579 * math.exp(0.0365*temps)
        wm = ew + (wmo-ew) * math.exp(-2.303*x)
        

    elif wmo > ed:
        z = 0.424 * ( 1.0 - (rhs/100.0) ** 1.7 ) + 0.0694 * math.sqrt(wss) * ( 1 - (rhs/100.0) ** 8.0)
        x = z * 0.0579 * math.exp(0.0365*temps)
        wm = ed + ( wmo - ed) * math.exp(-2.303*x)
        


    ffmcs = 59.5 * (250.0 - wm) / (147.2 + wm)
    if ffmcs > 101.0:
        ffmcs = 101.0
    if ffmcs < 0.0:
        ffmcs = 0

    return ffmcs

#function to calculate dmc(duff moisture code)
def _dmc(t,r,p,dmc_old, month):
    if t < -1.1:
        t = 1.1
    rk = 1.894 * (t + 1.1) * (100.0 - r) * cffdrs_el[month-1] * 0.0001
    pr = 0
    if p < 1.5:
        pr = dmc_old
    else:
        ra = p
        rw = 0.92 * ra - 1.27
        wmi = 20.0 + 280.0 / math.exp(0.023 * dmc_old)
        if dmc_old > 65:
            b = 6.2 * math.log(dmc_old) - 17.2
        elif dmc_old <= 65 and dmc_old > 33:
            b = 14.0 - 1.3 * math.log(dmc_old)
        else:
            b = 100.0 / (0.5 + 0.3 * dmc_old)
        wmr = wmi + 1000.0 * rw / (48.77 + b * rw)
        pr = 43.43 * (5.6348 - math.log(wmr - 20.))
    if pr < 0:
        pr = 0
    dmc = pr + rk
    if dmc < 0:
        dmc = 0
    return dmc


#function to calculate dc(drought code)
def _dc(temps, rains, dc_old, month):
    t = temps
    if t < -2.8:
        t = -2.81
    pe = (0.36 * (t + 2.8) + cffdrs_fl[month - 1]) / 2.0
    if pe < 0.0:
        pe = 0.0
    
    dr = dc_old
    if rains <= 2.801:
        dr = dc_old
    elif rains > 2.801:
        ra = rains
        rw = 0.83 * ra - 1.27
        smi = 800.0 * math.exp(-dc_old / 400.0);
        qr = smi + 3.937 * rw;
        dr = 400.0 * math.log(800.0 / qr)
                
    if dr < 0.0:
        dr = 0.0
    v = 0.36 * (t + 2.8) + cffdrs_fl[month - 1]
    if v < 0.0:
        v = 0.0
    dc = dr + v * 0.5
    if dc < 0.0:
        dc = 0.0
    return dc

#function to calculate isi(initial spread index)
def _isi(ffmcs, wspd):
    fm = 147.2 * (101.0 - ffmcs) / (59.5 + ffmcs)
    sf = 19.115 * math.exp(-0.1386 * fm) * (1.0 + (fm ** 5.31) / 4.93e07)
    isi = sf * math.exp(0.05039 * wspd)
    return isi


#function to calculate bui(buildup index)
def _bui(dmc, dc):
        if dmc <= (0.4 * dc):
            bui = 0.8 * dc * dmc / (dmc + 0.4 * dc)
        else:
            bui = dmc - (1. - 0.8 * dc / (dmc + 0.4 * dc)) * (0.92 + ((0.0114 * dmc) ** 1.7))
        if bui < 0:
            bui = 0.0
        if dmc == 0 and dc == 0:
            bui = 0.0
        return bui

#function to calculate fwi(fire weather index)
def _fwi(bui, isi):
    if bui > 80:
        bb = 0.1 * isi * (1000.0 / (25.0 + 108.64 / math.exp(0.023*bui)))
    else:
        bb = 0.1 * isi * (0.626 * (bui ** 0.809) + 2.0)
    if bb <= 1:
        fwi = bb
    else:
        fwi = math.exp(2.72 * ((0.434 * math.log(bb)) ** 0.647))
    return fwi

In [21]:
#this segment of the code is for demo input and output of the system

#dataframe to store data for all the days
result = pd.DataFrame(columns = ['Date', 'Temperature', 'Humidity', 'Wind Speed', 'Rain', 'FFMC', 'DMC', 'DC', 'ISI', 'BUI', 'FWI', 'Fire Prob.', 'Risk Message'])

#input for the date for the demo
date_entry = input('Enter a date in YYYY-MM-DD format')
year, month, day = map(int, date_entry.split('-'))
date1 = datetime.date(year, month, day)

#initializing the indices
ffmc_i = 85.0
dmc_i = 6.0
dc_i = 15.0
result.loc[0] = [date1 - datetime.timedelta(days = 1), None, None, None, None, ffmc_i, dmc_i, dc_i, None, None, None, None, None]
i = 1
c = 1


while i == 1:
    
    #taking input for the atmospheric data
    temp = float(input("Enter temperature in celcius: "))
    temp_f = (temp * 9.0 / 5.0) + 32
    hum = float(input("Enter percentage humidity: "))
    ws = float(input("Enter wind speed in mph: "))
    r = float(input("Enter percipitation in inches: "))
    
    month = date1.month
    
    #calculating the indices by calling the respective methods
    ffmc = _ffmc(temp_f, hum, ws, r, ffmc_i)
    dmc = _dmc(temp_f, hum, r, dmc_i, month)
    dc = _dc(temp_f, r, dc_i, month)
    isi = _isi(ffmc, ws)
    bui = _bui(dmc, dc)
    fwi = _fwi(bui, isi)
    
    #changing the ffmc_i, dmc_i, dc_i values to the current day values
    ffmc_i = ffmc
    dmc_i = dmc
    dc_i = dc
    
    d_list = np.array([temp, hum, ws, r, ffmc, dmc, dc, isi, bui, fwi]).reshape(1,-1)
    
    #predicting the probabitily for the given data
    y_pred = SVC_class.predict_proba(d_list)
    p = y_pred[0][0]
    if p > 0.80:
        m = "extremly high probability of fire"
    elif p > 0.60:
        m = "high probability of fire"
    elif p > 0.40:
        m = "moderate probability of fire"
    elif p > 0.20:
        m = "low probability of fire"
    else:
        m = "very low probability of fire"
    
    #storing the feature values and the label to the dataframe
    result.loc[c] = [date1, temp, hum, ws, r, ffmc, dmc, dc, isi, bui, fwi, p, m]
    print(result.loc[c])
    c += 1
    date1 = date1 + datetime.timedelta(days = 1)
    i = int(input("Enter 1 for next day, 0 to exit"))

#printing the final dataframe    
result

Enter a date in YYYY-MM-DD format2021-5-31
Enter temperature in celcius: 22
Enter humidity: 80
Enter wind speed: 3
Enter percipitation: 2
Date                              2021-05-31
Temperature                             22.0
Humidity                                80.0
Wind Speed                               3.0
Rain                                     2.0
FFMC                               65.843613
DMC                                 8.463267
DC                                    30.292
ISI                                 0.633739
BUI                                 9.965732
FWI                                 0.381595
Fire Prob.                          0.037769
Risk Message    very low probability of fire
Name: 1, dtype: object
Enter 1 for next day, 0 to exit1
Enter temperature in celcius: 29
Enter humidity: 70
Enter wind speed: 1.5
Enter percipitation: 0
Date                          2021-06-01
Temperature                         29.0
Humidity                            70.0
W

,Date,Temperature,Humidity,Wind Speed,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Fire Prob.,Risk Message
0,2021-05-30,None,None,None,None,85.000000,6.000000,15.000000,None,None,None,None,None
1,2021-05-31,22.0,80.0,3.0,2.0,65.843613,8.463267,30.292000,0.633739,9.965732,0.381595,0.037769,very low probability of fire
2,2021-06-01,29.0,70.0,1.5,0.0,80.093540,15.200244,48.852000,1.238507,17.099367,1.126711,0.643322,high probability of fire
3,2021-06-02,15.0,75.0,6.0,5.0,41.689936,13.174072,56.521026,0.064157,16.647521,0.051905,0.000006,very low probability of fire
4,2021-06-03,30.0,60.0,0.0,0.0,65.129208,22.346259,75.405026,0.5301,25.672455,0.564364,0.150153,very low probability of fire
5,2021-06-04,30.0,60.0,0.5,0.0,82.409228,31.518447,94.289026,1.537079,34.339673,3.46929,0.945576,extremly high probability of fire
